In [1]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import torch.nn.functional as F 
import matplotlib.pyplot as plt
import numpy as np
torch.__version__

'2.3.1+cpu'

In [ ]:
torchSeed = 10
torch.manual_seed(torchSeed)

In [7]:
transformTrain = transforms.Compose(
    [
        transforms.Resize([64, 64]),
        transforms.ToTensor()
    ]
)

In [44]:
dirPathTrain = r"C:\Users\terce\Documents\IC - Ultrasom\code\gans_data"
trainDataset = datasets.ImageFolder(dirPathTrain,  transform=transformTrain)
trainLoader = torch.utils.data.DataLoader(trainDataset , batch_size=24, shuffle=True)

## Classe da Rede Neural GAN (Gerador)

In [49]:
class Generate(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.linear1 = nn.Linear(100, 32)
        self.linear2 = nn.Linear(32, 64)
        self.linear3 = nn.Linear(64, 128)
        self.linear4 = nn.Linear(128, 4096)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.dropout(F.leaky_relu(self.linear1(x), 0.2))
        x = self.dropout(F.leaky_relu(self.linear2(x), 0.2))
        x = self.dropout(F.leaky_relu(self.linear3(x), 0.2))
        x = torch.tanh(self.linear4(x))
        x = x.view(x.shape[0], 64*64)
        return x

## Classe da Rede Neural GAN (Discriminador)

In [50]:
class Discriminate(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Linear(4096, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, 32)
        self.linear4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.view(x.shape[0], 64*64)
        x = self.dropout(F.leaky_relu(self.linear1(x), 0.2))
        x = self.dropout(F.leaky_relu(self.linear2(x), 0.2))
        x = self.dropout(F.leaky_relu(self.linear3(x), 0.2))
        x = self.linear4(x)
        return x

In [51]:
modelGenerate = Generate()
modelDiscriminate = Discriminate()

In [52]:
mGenOptimizer = optim.Adam(modelGenerate.parameters(), lr=0.001)
mDisOptimizer = optim.Adam(modelDiscriminate.parameters(), lr=0.001)

criterion = nn.BCEWithLogitsLoss()

In [53]:
device = torch.device('cpu')
modelGenerate.to(device)
modelDiscriminate.to(device)

Discriminate(
  (linear1): Linear(in_features=4096, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=32, bias=True)
  (linear4): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [ ]:
for epoch in range(15):
    GenLoss, DisLoss = 0., 0.

    for i, data in enumerate(trainLoader):
        
        inputs, _ = data
        batchSize = inputs.size(0)
        inputs = inputs.to(device)


        ### Treinamento do Gerador

        mGenOptimizer.zero_grad()

        #Criação do Ruído
        noise = np.random.uniform(low=-1., high=1., size=(batchSize, 100))
        noise = torch.from_numpy(noise).float().to(device)

        # Geração das Imagens
        fakeImages = modelGenerate.forward(noise)

        # Avalia a capacidade do model Gerador de criar imagens semelhantes as 
        # imagens reais (Ao ponto do modelo Discriminador ter a saída mais próxima de 1)
        fakeOutput = modelDiscriminate.forward(fakeImages)

        fakeLabel = torch.ones(24).to(device)

        GenLoss = criterion(fakeOutput.view(*fakeLabel.shape), fakeLabel)
        GenLoss.backward()
        mGenOptimizer.step()

        ### Treinamento do Discriminador

        # Treina o modelo Discriminador com a imagem atual
        realOutputs = modelDiscriminate.forward(inputs)
        realLabels = (torch.ones(batchSize) * 0.9).to(device)

        DisLossReal = criterion(realOutputs.view(*realLabels.shape), realLabels)
